<a href="https://colab.research.google.com/github/Cassio-4/superMarioRL/blob/master/DQN/dqn_mario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!rm -r logs

In [0]:
!pip install gym-super-mario-bros
!pip install import-ipynb
!pip install -q tf-nightly-2.0-preview
# Install the PyDrive wrapper & import libraries.
!pip install -U -q PyDrive

In [0]:
import import_ipynb
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
#importando wrapper
# https://drive.google.com/open?id=1hk5V6dPPgcAFNCqTlDwQRM_dlqdxCEwA
my_wrapper = drive.CreateFile({'id':'1hk5V6dPPgcAFNCqTlDwQRM_dlqdxCEwA'})
my_wrapper.GetContentFile('env_wrapper.ipynb')
import env_wrapper as wrappers
#importando modelo de rede dqn
#https://drive.google.com/open?id=14wjjrd_iI1y1x-N01PJaDBzqC43c1ipd
dqn_model = drive.CreateFile({'id':'14wjjrd_iI1y1x-N01PJaDBzqC43c1ipd'})
dqn_model.GetContentFile('dqn.ipynb')
import dqn as dqn_model

In [0]:
import time
import datetime
import numpy as np
import collections
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
from tensorflow import summary
%load_ext tensorboard

In [0]:
!nvidia-smi

In [0]:
DEFAULT_ENV_NAME = 'SuperMarioBros-v0'
# Reward boundary for the last 100 episodes to stop training. 
#MEAN_REWARD_BOUND = 19.5

# Valor de Gamma usado na aproximação de Bellman
GAMMA = 0.99
# O tamanho da 'batch' amostrada do Replay Buffer
BATCH_SIZE = 32
# Capacidade máxima do Replay Buffer
REPLAY_SIZE = 10000
# Número de frames que esperamos antes de começar a treinar para popular o Replay Buffer
REPLAY_START_SIZE = 10000
# Taxa de aprendizado usada pelo otimizador Adam, que é usado nesse código
LEARNING_RATE = 1e-4
# Frequência de sincronização dos pesos do modelo de treino para o modelo alvo.
SYNC_TARGET_FRAMES = 1000

""" Parâmetros de declínio de Epsilon. No começo do treinamento começamos com
Epsilon=1.0, ou seja, com ações totalmente aleatórias. Depois, durante os 100K
primeiros frames, Epsilon decai linearmente para 0.02,"""
EPSILON_DECAY_LAST_FRAME = 10**5
EPSILON_START = 1.0
EPSILON_FINAL = 0.02

In [0]:
"""
Código referente ao Replay Buffer. A cada Step no ambiente colocamos a 
transição no Buffer, mantendo apenas um número fixo de Steps, nesse caso 10k 
transições. Para treinamento, selecionamos aleatóriamente um lote (batch) de
transições do Replay Buffer, o que permite que quebremos a correlação entre 
passos subsequentes no ambiente.
"""
Experience = collections.namedtuple('Experience', field_names=['state', 'action', 'reward', 'done', 'new_state'])

class ExperienceBuffer:
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, dones, next_states = zip(*[self.buffer[idx] for idx in indices])
        return np.array(states), np.array(actions), np.array(rewards, dtype=np.float32), \
               np.array(dones, dtype=np.uint8), np.array(next_states)

""" 
Agente:interage com o ambiente e salva o resultado da interação no 
experience replay buffer
"""
class Agent:
    """
    Na inicialização do Agente guardamos referências para o ambiente (env) e 
    para o experience replay buffer, registrando a observação(observation) atual
    e a recompensa total acumulada até então.
    """
    def __init__(self, env, exp_buffer):
        self.env = env
        self.exp_buffer = exp_buffer
        self._reset()

    def _reset(self):
        self.state = env.reset()
        self.total_reward = 0.0
    """
    Faz um step no ambiente e guarda o resultado no Buffer. Com probabilidade 
    Epsilon tomamos uma ação aleatória, caso contrário utilizamos o modelo para 
    obter os Q-values para todas as possíveis ações e escolhemos a melhor.
    """
    def play_step(self, net, epsilon=0.0, device="cpu"):
        done_reward = None

        if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            state_a = np.array([self.state], copy=False)
            state_v = torch.tensor(state_a).to(device)
            q_vals_v = net(state_v)
            _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())
        """
        Passamos a ação escolhida para o ambiente e pegamos a próxima observation
        e recompensa, guardamos os dados no experience buffer e tratamos o fim-
        -de-episodio. O resultado dessa função é a recompensa total acumulada se
        chegamos ao fim-de-episodio com esse step ou None caso contrário.
        """
        new_state, reward, is_done, _ = self.env.step(action)
        self.total_reward += reward

        exp = Experience(self.state, action, reward, is_done, new_state)
        self.exp_buffer.append(exp)
        self.state = new_state
        if is_done:
            done_reward = self.total_reward
            self._reset()
        return done_reward

"""
Função que calcula a Loss para a batch amostrada.
Argumentos: a batch como uma tupla de arrays (método sample() do experience 
buffer), nossa rede de treino e a rede alvo, periodicamente sincronizada com a 
de treino.
"""
def calc_loss(batch, net, tgt_net, device="cpu"):
    states, actions, rewards, dones, next_states = batch
    """
    Empacota arrays individuais NumPy com dados do lote(batch) em tensores
    PyTorch e copia para a GPU
    """
    states_v = torch.tensor(states).to(device)
    next_states_v = torch.tensor(next_states).to(device)
    actions_v = torch.tensor(actions).to(device)
    rewards_v = torch.tensor(rewards).to(device)
    done_mask = torch.ByteTensor(dones).to(device)
    """
    Passa as observations para a primeira rede e extrai os Q-Values específicos
    para as ações tomadas (?) usando a operação de tensor gather().
    """
    state_action_values = net(states_v).gather(1, actions_v.unsqueeze(-1)).squeeze(-1)
    """
    Aplica a rede alvo para as observações de próximo estado e calcula o máximo 
    Q-Value ao longo da mesma dimensão 1 de ação(?). A função max() retorna 
    ambos os valores máximos e seus índices (max e argmax), entretanto neste 
    caso, estamos apenas interesados nos valores, então pegamos a primeira 
    entrada do resultado
    """
    next_state_values = tgt_net(next_states_v).max(1)[0]
    """
    Aqui há um ponto simples porém, importante: se a transição no batch é do 
    último step do episódio, então nosso valor da ação não tem uma recompensa com
    desconto do próximo estado, visto que não há próximo estado de onde tirar 
    uma recompensa. Pode parecer pequeno mas é muito importante na prática, sem 
    isso o treino NÃO irá convergir
    """
    next_state_values[done_mask] = 0.0
    next_state_values = next_state_values.detach()
    """
    Calcula o valor da aproximação de Bellman e o erro quadrático médio (MSE)
    (loss)
    """
    expected_state_action_values = next_state_values * GAMMA + rewards_v
    return nn.MSELoss()(state_action_values, expected_state_action_values)

In [0]:
import traceback
import warnings
import sys

def warn_with_traceback(message, category, filename, lineno, file=None, line=None):

    log = file if hasattr(file,'write') else sys.stderr
    traceback.print_stack(file=log)
    log.write(warnings.formatwarning(message, category, filename, lineno, line))

warnings.showwarning = warn_with_traceback

In [0]:
if __name__ == "__main__":
    # Se cuda estiver disponível setamos device como cuda
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    """
    Cria ambiente (env) com todos os empacotadores aplicados, a rede neural que 
    será treinada e a rede alvo com a mesma arquitetura. No início serão 
    inicializadas com pesos diferentes, mas isso não importa visto que serão 
    sincronizadas a cada SYNC_TARGET_FRAMES, o que corresponde aproximadamente
    a um episódio de Pong (quantos frames tem um episódio de Mario?)
    """
    env = wrappers.make_env(DEFAULT_ENV_NAME)
    net = dqn_model.DQN(env.observation_space.shape, env.action_space.n).to(device)
    tgt_net = dqn_model.DQN(env.observation_space.shape, env.action_space.n).to(device)
    
    current_time = str(datetime.datetime.now())
    train_log_dir = 'logs/' + current_time
    writer = summary.create_file_writer(train_log_dir)
    #writer = SummaryWriter(comment="-" + DEFAULT_ENV_NAME)
    """
    Cria o experience replay buffer de tamanho REPLAY_SIZE e o passa para o agente.
    Epsilon é inicializado com 1.0, mas decaira a cada iteração.
    """
    buffer = ExperienceBuffer(REPLAY_SIZE)
    agent = Agent(env, buffer)
    epsilon = EPSILON_START
    
    """
    A última parte antes do loop de treino consiste em: criar o otimizador, um 
    buffer para recompensas de episodios completos (full episode rewards(?)), um
    contador de frames e variaveis para rastrear velocidade, e a melhor 
    recompensa média alcançada.Toda vez que a recompensa média alcançada for 
    maior que o último recorde, salvamos o modelo em um arquivo.
    """

    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
    total_rewards = []
    frame_idx = 0
    ts_frame = 0
    ts = time.time()
    best_mean_reward = None

    while True:
        """
        No início do loop de treino, contamos o número de iterações completadas e 
        decrescemos Epsilon de acordo com o planejado. Epsilon decairá linearmente 
        durante o número de frames dados (EPSILON_DECAY_LAST_FRAME) e será mantido
        no mesmo nível que EPSILON_FINAL.
        """
        frame_idx += 1
        epsilon = max(EPSILON_FINAL, EPSILON_START - frame_idx / EPSILON_DECAY_LAST_FRAME)
        """
        Faz o agente dar um único step (usando a rede atual e o valor de epsilon).
        Essa função não retorna None apenas se este Step for o último do episódio,
        nesse caso, relatamos nosso progresso. Especificamente, calcula-se e 
        mostra tanto no console quanto no TensorBoard, esses valores:
        - Velocidade (speed) como uma contagem de frames processados /s
        - Contagem de episódios reproduzidos
        - Média de recompensa para os últimos 100 episódios
        - Valor atual de Epsilon
        """
        reward = agent.play_step(net, epsilon, device=device)
        if reward is not None:
            total_rewards.append(reward)
            speed = (frame_idx - ts_frame) / (time.time() - ts)
            ts_frame = frame_idx
            ts = time.time()
            mean_reward = np.mean(total_rewards[-100:])
            print("%d: done %d games, mean reward %.3f, eps %.2f, speed %.2f f/s" % (
                frame_idx, len(total_rewards), mean_reward, epsilon,
                speed
            ))
            with writer.as_default():
                tf.summary.scalar("epsilon", epsilon, frame_idx)
                tf.summary.scalar("speed", speed, frame_idx)
                tf.summary.scalar("reward_100", mean_reward, frame_idx)
                tf.summary.scalar("reward", reward, frame_idx)

                #writer.("epsilon", epsilon, frame_idx)
                #writer.add_scalar("speed", speed, frame_idx)
                #writer.add_scalar("reward_100", mean_reward, frame_idx)
                #writer.add_scalar("reward", reward, frame_idx)
            """
            Toda vez que a recompensa média para os últimos 100 episódios atingir
            um máximo, relatamos e salvamos os parâmetros do modelo. 
            (Para PONG)Se a esta ultrapassar o limite paramos o treino, para Pong
            o limite é 19.5 que significa ganhar mais de 19 jogos dos 21 possíveis.
            """
            if best_mean_reward is None or best_mean_reward < mean_reward:
                torch.save(net.state_dict(), DEFAULT_ENV_NAME + "-best.dat")
                if best_mean_reward is not None:
                    print("Best mean reward updated %.3f -> %.3f, model saved" % (best_mean_reward, mean_reward))
                best_mean_reward = mean_reward
            if mean_reward > 3000:
                print("Solved in %d frames!" % frame_idx)
                break
        """
        Checa se o Replay Buffer é grande o bastante para começar o treino. No
        começo devemos esperar REPLAY_SIZE transições. A condição seguinte 
        sincroniza os parâmetros da rede principal e da rede alvo a cada SYNC_TARGET_FRAMES.
        """
        if len(buffer) < REPLAY_START_SIZE:
            continue
        if frame_idx % SYNC_TARGET_FRAMES == 0:
            tgt_net.load_state_dict(net.state_dict())
        """
        A última parte do loop de treino é simples, mas requer a maior parte 
        para executar: zera os gradientes, amostra lotes de dados do experience 
        replay buffer, calcula perca (loss) e faz o passo de otimização para 
        minimizar a perca.
        """
        optimizer.zero_grad()
        batch = buffer.sample(BATCH_SIZE)
        loss_t = calc_loss(batch, net, tgt_net, device=device)
        loss_t.backward()
        optimizer.step()
    writer.close()